In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install 'typing-extensions<4.6.0' --force-reinstall
!pip install cryptocompare
!pip install alpaca-py

  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.10.0
    Uninstalling typing_extensions-4.10.0:
      Successfully uninstalled typing_extensions-4.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy 2.0.27 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
pydantic 2.6.3 requires typing-extensions>=4.6.1, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.16.3 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for cryptocompare: filename=cryptocompare-0.7.6-py3-none-any.whl size=6289 sha256=28336a9ddbc8ffade6e43e01e76039e93608a0529cf88ddb5b0021aa00ecffa5
  Stored in directory: /root/.cache/pip/wheels/97/81/6a/261bc499b9d9bf8b391

In [ ]:
import pandas as pd
import datetime as datetime
import cryptocompare
import csv
import numpy as np
import random
from collections import deque
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, load_model
from alpaca.trading.client import TradingClient
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Concatenate, Attention
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

In [ ]:
cryptocompare.cryptocompare._set_api_key_parameter("8632283f7e59e6054da95d4fc2728404b2931eee776abce0d69d86af35a86e54")

#The limit to request is 999 hours
data = cryptocompare.get_historical_price_hour('ETH', 'USD', limit=999, exchange='CCCAGG', toTs=datetime.datetime.now())

#Removing unwanted values
headers_to_exclude = ['conversionType', 'conversionSymbol']
cleaned_data = [{k: v for k, v in item.items() if k not in headers_to_exclude} for item in data]

csv_file = "/content/eth_1_year_data.csv"

#Writing data to the csv
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=cleaned_data[0].keys())
    writer.writeheader()
    writer.writerows(cleaned_data)
file.close()

#Using data from the CSV to calculate sma and adding it to the csv
df = pd.read_csv('/content/eth_1_year_data.csv')
df['50sma'] = df['close'].rolling(50).mean()
df['20sma'] = df['close'].rolling(20).mean()

# Drop rows with NaN values after calculating SMAs
df.dropna(inplace=True)

df.to_csv(csv_file)

In [ ]:
# Load data
csv_file = "/content/eth_1_year_data.csv"
df = pd.read_csv(csv_file)

# Assuming 'close' is the feature we want to predict
# Let's also include 'high', 'low', 'open', 'volumefrom', 'volumeto', '20sma', and '50sma' as features
features = ['high', 'low', 'open', 'volumefrom', 'volumeto', 'close', '20sma', '50sma']
df[features] = df[features].fillna(method='ffill')  # Forward fill for missing values

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[features].values)

# Function to create sequences
def create_sequences(data, time_steps=20):
    X, y = [], []
    for i in range(time_steps, len(data)):
        X.append(data[i-time_steps:i])
        y.append(data[i, 0])  # Assuming 'close' price is at index 5
    return np.array(X), np.array(y)

# Create sequences
time_steps = 20  # Number of time steps you're looking back to predict the future
X, y = create_sequences(scaled_data, time_steps)

# Split the data
split = int(0.8 * len(X))  # 80% for training and 20% for testing
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# LSTM expects input to be in the shape of [samples, time steps, features]
input_shape = (X_train.shape[1], X_train.shape[2])

# Define the LSTM model
def lstm_model(input_shape):
    inputs = Input(shape=input_shape)
    lstm_out, hidden_state, cell_state = LSTM(50, activation='relu', return_sequences=True, return_state=True)(inputs)
    lstm_out = Dropout(0.2)(lstm_out)
    query_value_attention_seq = Attention()([lstm_out, lstm_out])
    query_value_attention_concat = Concatenate(axis=-1)([lstm_out, query_value_attention_seq])
    lstm_out_2 = LSTM(50, activation='relu', return_sequences=False)(query_value_attention_concat)
    lstm_out_2 = Dropout(0.2)(lstm_out_2)
    output = Dense(1, activation='linear')(lstm_out_2)
    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer=Adam(), loss='mean_squared_error')
    return model

# Build the model
attention_lstm_model = lstm_model(input_shape)

# Train the model
history = attention_lstm_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Save the trained LSTM model
attention_lstm_model.save('attention_lstm_model.h5')

# Evaluate the model
loss = attention_lstm_model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {loss}')

Epoch 1/100
24/24 [==============================] - 16s 84ms/step - loss: 0.0368 - val_loss: 0.0183
Epoch 2/100
24/24 [==============================] - 1s 62ms/step - loss: 0.0047 - val_loss: 0.0036
Epoch 3/100
24/24 [==============================] - 2s 64ms/step - loss: 0.0033 - val_loss: 0.0027
Epoch 4/100
24/24 [==============================] - 2s 71ms/step - loss: 0.0030 - val_loss: 0.0018
Epoch 5/100
24/24 [==============================] - 2s 65ms/step - loss: 0.0030 - val_loss: 0.0018
Epoch 6/100
24/24 [==============================] - 2s 97ms/step - loss: 0.0025 - val_loss: 0.0041
Epoch 7/100
24/24 [==============================] - 3s 104ms/step - loss: 0.0023 - val_loss: 0.0018
Epoch 8/100
24/24 [==============================] - 2s 63ms/step - loss: 0.0025 - val_loss: 0.0020
Epoch 9/100
24/24 [==============================] - 1s 62ms/step - loss: 0.0021 - val_loss: 0.0045
Epoch 10/100
24/24 [==============================] - 2s 63ms/step - loss: 0.0021 - val_loss: 0.00

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# After training, use the most recent data to make a prediction
last_sequence = X_test[-1].reshape((1, time_steps, X_test.shape[2]))
predicted_price_scaled = attention_lstm_model.predict(last_sequence)

# Invert the scaling for the predicted price
predicted_price = scaler.inverse_transform(np.concatenate((predicted_price_scaled, np.zeros((predicted_price_scaled.shape[0], len(features)-1))), axis=1))[:,0]

# Fetch the current price of ETH from CryptoCompare
current_price_data = cryptocompare.get_price('ETH', currency='USD')
current_price = current_price_data['ETH']['USD']

# Calculate the prediction error
percentage_error = abs(current_price - predicted_price) / current_price * 100

# Output the results
print(f"Predicted ETH Price: {predicted_price[0]} USD")
print(f"Current ETH Price: {current_price} USD")
print(f"Prediction Error: {percentage_error[0]:.2f}%")

1/1 [==============================] - 0s 352ms/step
Predicted ETH Price: 3238.272385774851 USD
Current ETH Price: 3424.2 USD
Prediction Error: 5.43%


In [ ]:
# Initialize your API keys (these should be kept secret and not hardcoded in production)
trading_client = TradingClient('', '', paper=True)

In [ ]:
class MarketEnvironment:
    def __init__(self, csv_file_path, initial_balance=100000, lookback_window_size=20):
        self.data = pd.read_csv("/content/eth_1_year_data.csv")
        self.initial_balance = initial_balance
        self.lookback_window_size = lookback_window_size
        self.state_size = (lookback_window_size, self.data.shape[1] + 2)
        self.balance = initial_balance
        self.holdings = 0
        self.total_portfolio_value = self.balance
        self.done = False
        self.current_step = 0
        self.alpaca_api = trading_client

    def reset(self):
        self.balance = self.initial_balance
        self.holdings = 0
        self.total_portfolio_value = self.balance
        self.current_step = 0
        self.done = False
        return self.get_state(self.current_step)

    def step(self, action):
        assert action in [0, 1, 2]
        self._update_portfolio()
        current_price = self._get_current_price()
        reward = 0
        percent_of_balance = self.balance * 0.01

        if action == 1:  # Buy
            qty_to_buy = percent_of_balance / current_price
            reward = self._buy(current_price, qty_to_buy)
        elif action == 2 and self.holdings > 0:  # Sell
            qty_to_sell = min(self.holdings, percent_of_balance / current_price)
            reward = self._sell(current_price, qty_to_sell)

        self.current_step += 1
        if self.current_step >= len(self.data):
            self.done = True

        next_state = self.get_state(self.current_step)
        return next_state, reward, self.done, {}

    def _update_portfolio(self):
        account_info = self.alpaca_api.get_account()
        self.balance = float(account_info.cash)
        # self.holdings update logic here if applicable

    def _get_current_price(self):
        price = cryptocompare.get_price('ETH', currency='USD')['ETH']['USD']
        return price

    def _buy(self, current_price, qty):
        try:
            market_order_data = MarketOrderRequest(
                symbol="ETHUSD",
                qty=qty,
                side=OrderSide.BUY,
                time_in_force=TimeInForce.GTC
            )
            market_order = self.alpaca_api.submit_order(order_data=market_order_data)
            print("Buy order submitted successfully:", market_order)
            # Update balance and holdings assuming the order is filled immediately
            self.balance -= qty * current_price
            self.holdings += qty
            return qty * current_price  # or another reward calculation
        except Exception as e:
            print(f"An error occurred while submitting the buy order: {e}")
            return 0  # No reward if the order fails

    def _sell(self, current_price, qty):
        try:
            market_order_data = MarketOrderRequest(
                symbol="ETHUSD",
                qty=qty,
                side=OrderSide.SELL,
                time_in_force=TimeInForce.GTC
            )
            market_order = self.alpaca_api.submit_order(order_data=market_order_data)
            print("Sell order submitted successfully:", market_order)
            # Update balance and holdings assuming the order is filled immediately
            self.balance += qty * current_price
            self.holdings -= qty
            return qty * current_price  # or another reward calculation
        except Exception as e:
            print(f"An error occurred while submitting the sell order: {e}")
            return 0  # No reward if the order fails

    def get_state(self, current_step):
        window_start = max(current_step - self.lookback_window_size, 0)
        window_end = current_step + 1
        window_data = self.data.iloc[window_start:window_end].values

        balance_and_holdings = np.array([[self.balance, self.holdings]] * len(window_data))

        state = np.concatenate((window_data, balance_and_holdings), axis=1)
        return state

In [ ]:
class TradingAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)  # Memory for experience replay
        self.gamma = 0.95  # Discount rate for future rewards
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.01  # Minimum exploration rate
        self.epsilon_decay = 0.995  # Decay rate for exploration probability
        self.learning_rate = 0.001  # Learning rate
        self.model = self._build_model()  # The DQN model
        self.lstm_model = load_model('attention_lstm_model.h5')

    def _build_model(self):
        """Builds a NN model."""
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        """Stores experiences in memory."""
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state, current_price, use_lstm=True):
        # DQN model part for action selection
        act_values = self.model.predict(state)
        if random.uniform(0, 1) <= self.epsilon:
            # Exploration
            dqn_action = random.randrange(self.action_size)
        else:
            # Exploitation
            dqn_action = np.argmax(act_values[0])

        if use_lstm:
            lstm_input_shape = (1, 20, 8)  # LSTM expects (batch_size, time_steps, features)
            lstm_input = np.zeros(lstm_input_shape)

            # Make prediction with the LSTM model
            lstm_prediction = self.lstm_model.predict(lstm_input)[0][0]

            # Interpret LSTM prediction
            # Adjust the Q-values based on LSTM's prediction
            # Example: Increase sell action's Q-value if prediction is lower than current price
            predicted_change = lstm_prediction - current_price
            significance_threshold = 0.01  # Example threshold

            # Adjusting DQN's Q-values based on LSTM prediction
            if predicted_change < -significance_threshold:
                # If predicted price is significantly lower, increase the Q-value for sell action
                act_values[0][2] += abs(predicted_change)  # Assuming action 2 is 'sell'
            elif predicted_change > significance_threshold:
                # If predicted price is significantly higher, increase the Q-value for buy action
                act_values[0][1] += abs(predicted_change)  # Assuming action 1 is 'buy'

            # Now, select action based on possibly adjusted Q-values
            final_action = np.argmax(act_values[0])
        else:
            final_action = dqn_action

        return final_action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            if reward is None:
                print("Warning: Received None as reward")
                continue  # or handle this case as appropriate for your situation
            target = reward
            next_state_pred = self.model.predict(next_state)
            if next_state_pred is not None and len(next_state_pred) > 0:
                max_next_state_pred = np.amax(next_state_pred[0])
                target += self.gamma * max_next_state_pred
            else:
                print("Invalid prediction for next_state:", next_state_pred)
                continue  # or handle as appropriate
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        """Loads a saved model."""
        self.model.load_weights(name)

    def save(self, name):
        """Saves the model."""
        self.model.save_weights(name)

In [ ]:
# MAIN LOOP

# Initialize the market environment
initial_balance = 100000  # Set an initial balance, for example, $100,000
lookback_window_size = 1  # The number of past time points to consider for the state

# Initialize the market environment
market_environment = MarketEnvironment(data, initial_balance, lookback_window_size)

# Define the size of the state and action space
state_size = market_environment.state_size[0] * market_environment.state_size[1]
action_size = 3  # Assuming three actions: [hold, buy, sell]

# Initialize the trading agent
trading_agent = TradingAgent(state_size, action_size)

# How many episodes we want the agent to train for
num_episodes = 150 # Tweak according to performance

# How many timesteps per episode
timesteps_per_episode = len(data) - lookback_window_size
print(f"timesteps_per_episode: {timesteps_per_episode}") # Currently 999 hours or 41.625 days of historical data

# Batch size for experience replay
batch_size = 32 # 32 is a commonly used default because it's been found to be a good trade-off between training speed and model update stability

for e in range(num_episodes):
    # Reset the market environment at the start of each episode
    state = market_environment.reset()

    # Flatten the state to feed into the neural network
    state = np.reshape(state, (1, -1))  # This will ensure the state is a 2D array with the correct shape

    for time in range(timesteps_per_episode):
        # The agent takes action
        action = trading_agent.act(state, current_price)

        # Apply the action to the market environment to get the next state and reward
        next_state, reward, done, _ = market_environment.step(action)

        # Remember the previous state, action, reward, and next state
        trading_agent.remember(state, action, reward, next_state, done)

        # Make the next_state the new current state for the next frame.
        state = next_state

        # If done, exit the loop
        if done:
            print(f"Episode: {e + 1}/{num_episodes}, Total Portfolio Value: {market_environment.total_portfolio_value}, P/L: {market_environment.total_portfolio_value - initial_balance}")
            break

        # Train the agent with experiences in replay memory
        if len(trading_agent.memory) > batch_size:
            trading_agent.replay(batch_size)

    # Optionally, you can save the model every X episodes
    if e % 10 == 0:
        trading_agent.save(f"trading_agent_{e}.h5")

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - ETA: 0s

In [ ]:
"""
MODEL TIMING

The total time per episode in seconds would be 0.024 seconds * 999 timesteps = 23.976 seconds.

For 150 episodes, the total time would be 23.976 seconds/episode * 150 episodes = 3596.4 seconds.

Converting this to hours: 3596.4 seconds / 3600 seconds/hour ≈ 0.999 hours.
"""

In [ ]:
"""
HYPERPARAMETERS

Time Steps (time_steps): Used in the sequence creation function, it defines the number of historical data points the LSTM model will consider for predicting the next value. It's set to 20, meaning the model looks at the last 20 data points to make a prediction.

Data Split (split): Determines the ratio of training to testing data. It's set to 80% for training and the remaining 20% for testing.

LSTM Units: Inside the lstm_model function, LSTM layers with 50 units are defined, which signifies the dimensionality of the output space.

Dropout Rate: Dropout layers with a rate of 0.2 are used to prevent overfitting by randomly setting a fraction of input units to 0 at each update during training.

Dense Layer Units: The dense layers in the LSTM model have 24 units each, which is another hyperparameter defining the output dimension of the layer.

Batch Size for Training LSTM Model (batch_size): The LSTM model is trained with a batch size of 32, which means 32 sequences are passed through the network at once.

Epochs for Training LSTM Model: The model is trained for 100 epochs, where one epoch means that the entire dataset is passed forward and backward through the neural network once.

Lookback Window Size (lookback_window_size): Set to 20, this defines how many past observations the trading environment will return as the state.

State Size (state_size): Calculated based on the lookback_window_size and the number of features in the data. It represents the shape of the input that the trading agent will receive.

Gamma (gamma): This is the discount factor for future rewards in the DQN agent, set to 0.95.

Epsilon (epsilon): The exploration rate for the agent, starting at 1.0 (100% exploration).

Epsilon Minimum (epsilon_min): The minimum exploration rate, set to 0.01.

Epsilon Decay (epsilon_decay): The rate at which the exploration rate decays, set to 0.995.

Learning Rate (learning_rate): For the Adam optimizer in the DQN, it's set to 0.001.

Number of Episodes (num_episodes): The number of episodes for training the DQN model. It's initially set to 1800, which indicates how many times the training process will iterate over the entire dataset.

Timesteps Per Episode (timesteps_per_episode): The number of steps the agent will take in the environment per episode. It's initially logged as 999, which is derived from the available historical price hours minus the lookback_window_size.
"""